# CASSIS data preprocessing: convert to pickled dataframes

In [143]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from astropy.io import fits
from astropy.table import Table
from IPython.core.debugger import set_trace as st

In [144]:
# Some useful functions....
idx_unravel = np.loadtxt('cassis_unravel_indices.txt', delimiter=',', dtype='int')

def convert_fits_to_pickle(path, verify_pickle=False, verbose=False):
    """Full conversion from CASSIS <filename.fits to <filename>.pkl, which contains a pd.DataFrame.
    
    Args:
        path (str): Path to <filename>.fits file (of a CASSIS observation).
        verify_pickle (bool): Confirm the pickle was succesful created; does so by comparing the
            pd.DataFrame before and after writing the pickle.
        verbose (bool): Whether to flood STDOUT with a bit of info or not.
        
    Returns:
        pickle_path (str): Path to saved .pkl file.
        df (pd.DataFrame): Pandas dataframe with appropriate labels (wavelength, flux, etc.).
        header (astropy.io.fits.header.Header): Information about observation from telescope.
        
    Note:
        DataFrame can be retrieved from the pickle by, e.g., df = pd.read_pickle(pickle_path).
    """
    
    if verbose:
        print('Pickling: ', path)
    
    # Convert .fits file to pandas DataFrame, header.Header object.
    df, header = cassis_fits_to_dataframe(path)

    # Determine the pickle_path to save to. Being explicit here to 'pickle_path' is clear.
    base_filename = path.replace('.fits', '.pkl').split('/')[-1]
    
    # Save the dataframe to a pickle.
    pickle_path = 'cassis_dataframes/' + base_filename
    df.to_pickle(pickle_path)
    
    if verbose:
        print('...saved: ', pickle_path)

    # Test dataframes for equality before/after pickling if verify_pickle == True.
    if verify_pickle:
        tmp_df = pd.read_pickle(pickle_path)
        if df.equals(tmp_df):
            if verbose:
                print()
                print('DataFrame integrity verified -- pickling went OK!')
                print()
        else:
            raise ValueError('Dataframes not equal before/after pickling!')
    
    return pickle_path, df, header


def cassis_fits_to_dataframe(path):
    """Take a CASSIS .fits file, return a pandas DataFrame containing the data (with labels) and astropy header.
    
    Args:
        path (str): Path of the .fits file (assumed to be a CASSIS observation file).
        
    Returns:
        df (pd.DataFrame): Pandas dataframe with appropriate labels (wavelength, flux, etc.).
        header (astropy.io.fits.header.Header): Information about observation from telescope.
        
    Note:
        Header can be manipulated with, e.g., header.totextfile(some_path).
        See http://docs.astropy.org/en/stable/io/fits/api/headers.html.
    """
    
    # Read in .fits file.
    hdu = fits.open(path)
    
    # Retrieve the header object.
    header = hdu[0].header
    
    # Extract column labels/descriptions from header.
    hdu_keys = [header['col'+str(x).zfill(2)+'def'] for x in range(1,16)]
    hdu_keys.append('unknown')  # Missing label? 16 columns, 15 labels...! Probably just 'flag4'.
    
    # Convert data to pandas DataFrame.
    dtable = Table(test_hdu[0].data)
    dtable = dtable[idx_unravel]  # Needed to reorder the wavelength grid such that it's monotonic.
    df = dtable.to_pandas()
    
    # Convert the nondescriptive column labels (e.g., 'col01def', 'col02def') to descriptive labels.
    old_keys = list(df.keys())
    new_keys = hdu_keys
    mydict = dict(zip(old_keys, new_keys))
    df = df.rename(columns=mydict)  # Renamed DataFrame columns here.
    
    return df, header

***

## Find out how many files we're working with

In [165]:
spec_dir = 'spectra/'
spec_files = np.sort(glob.glob(spec_dir + '*.fits'))

In [166]:
len(spec_files)

11081

In [167]:
meta_dir = 'metadata/'
meta_files = np.sort(glob.glob(meta_dir + '*.tbl'))

In [168]:
len(meta_files)

17322

More metadata files than spectrum files because some observations failed.

## Convert spectra to dataframes, save as pickles and save metadata

###### Creates meta_dataframe.pkl and cassis_dataframes/_______.pkl * a billion.

In [169]:
meta_head = ['aorkey', 'object_name', 'ra', 'dec', 'flux_units', 'file_path']

def record_metadata(metadata, pickle_path, df, header):
    file_path = pickle_path
    
    # Can query header
    ra = header['RA_SLT']  # Right ascension.
    dec = header['DEC_SLT']  # Declination.
    flux_units = header['BUNIT']  # Units of flux (typically Jy)
    aorkey = header['AORKEY']  # Unique observation identifier (specific to Spitzer Space Telescope).
    object_name = header['OBJECT']  # Object name.    
    
    record = (aorkey, object_name, ra, dec, flux_units, file_path)
    metadata.append(record)
    
    return metadata

def write_metadata_pickle(metadata):
    dfx = pd.DataFrame(metadata, columns=meta_head)
    dfx.to_pickle('meta_dataframe.pkl')
    return

In [170]:
perform_conversion = False

In [171]:
# Note the break I've added; remove for full conversion.
if perform_conversion:
    metadata = []
    
    print('=============================\nConverting fits files...\n=============================\n')

    # Iterate over all the fits files and convert them.
    for index, fits_file in enumerate(spec_files):
#         if index >= 2:
#             break

        if index % 500 == 0:
            print(index, '/', len(spec_files))

        pickle_path, df, header = convert_fits_to_pickle(fits_file, verify_pickle=True, verbose=False)
        metadata = record_metadata(metadata, pickle_path, df, header)

    print('\n=============================\nComplete.\n=============================')
    
    dfx = write_metadata_pickle(metadata)

In [172]:
dfx.head()

,aorkey,object_name,ra,dec,flux_units,file_path
0,10019072,HD 99754,172.09622,-23.829099,Jy,cassis_dataframes/10019072.pkl
1,10019328,HR 5467,219.56055,54.021404,Jy,cassis_dataframes/10019328.pkl
2,10019584,HD 173511,280.41377,61.546739,Jy,cassis_dataframes/10019584.pkl
3,10019840,alp Lac,337.82051,50.284642,Jy,cassis_dataframes/10019840.pkl
4,10020096,HR 6348,255.31540,60.647838,Jy,cassis_dataframes/10020096.pkl


***

***

***

# Appendix A -- Example transformation from .fits to pd.dataframe

#### Convert spectrum file to dataframe, header

In [89]:
# Grab the first file from the glob list.
test_spec = spec_files[0]
test_spec

'spectra/10019072.fits'

In [90]:
# Read it in with astropy.io.fits, check dimensions.
test_hdu = fits.open(test_spec)
test_hdu.info()

Filename: spectra/10019072.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      72   (16, 387)   float32   


In [91]:
# Utilize our defined function to transform a string of the .fits filename to a pandas dataframe and header.
# 'header' will be an astropy.io.fits.header.Header object; see a couple subsections below for conversion options.
df, header = cassis_fits_to_dataframe(test_spec)

In [104]:
# monotonically_increasing(df['wavelength'])

In [105]:
# cassis_wave = np.array(df['wavelength'])
# np.savetxt('cassis_wavelength_grid.txt', cassis_wave, delimiter=',')

#### Inspect dataframe

In [312]:
df.shape

(387, 16)

In [313]:
df.head()

,wavelength,flux,error (RMS+SYS),error (RMS),error (SYS),offset uncertainty (CAL),sky,sky error,module,order,nod,status,flag1,flag2,flag3,unknown
0,7.57612,1.842944,0.027002,0.022974,0.004027,0.016090,0.0,0.711246,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,7.63660,1.771911,0.026625,0.022145,0.004480,0.018425,0.0,0.711246,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,7.69709,1.713980,0.022441,0.020821,0.001620,0.020722,0.0,0.711246,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,7.75757,1.661355,0.022177,0.019700,0.002477,0.018943,0.0,0.711246,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,7.81805,1.611652,0.025649,0.019104,0.006545,0.015696,0.0,0.711246,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [314]:
df.describe()

,wavelength,flux,error (RMS+SYS),error (RMS),error (SYS),offset uncertainty (CAL),sky,sky error,module,order,nod,status,flag1,flag2,flag3,unknown
count,387.000000,387.000000,387.000000,387.000000,387.000000,387.000000,387.0,387.000000,387.000000,387.000000,387.0,387.0,387.000000,387.0,387.0,387.0
mean,16.045664,0.979526,0.019800,0.014002,0.005798,0.007546,0.0,0.582891,1.431525,1.537468,1.0,0.0,0.310078,0.0,0.0,0.0
std,9.002149,0.912483,0.024442,0.012668,0.013877,0.004969,0.0,0.271884,1.118538,0.627947,0.0,0.0,0.691905,0.0,0.0,0.0
min,5.247490,0.083014,0.001760,0.001593,0.000000,0.001294,0.0,0.000000,0.000000,1.000000,1.0,0.0,0.000000,0.0,0.0,0.0
25%,8.150715,0.256977,0.006559,0.004582,0.001098,0.003116,0.0,0.707154,0.000000,1.000000,1.0,0.0,0.000000,0.0,0.0,0.0
50%,13.624480,0.605169,0.015639,0.013047,0.002661,0.006067,0.0,0.707294,1.000000,1.000000,1.0,0.0,0.000000,0.0,0.0,0.0
75%,21.266890,1.472475,0.028940,0.019901,0.006110,0.011882,0.0,0.709979,2.000000,2.000000,1.0,0.0,0.000000,0.0,0.0,0.0
max,37.355492,3.287086,0.375687,0.135014,0.240674,0.029590,0.0,0.723832,3.000000,3.000000,1.0,0.0,2.000000,0.0,0.0,0.0


#### Header from the .fits file

In [185]:
type(header)

astropy.io.fits.header.Header

In [326]:
# Uncomment below to see full header of one file as an example.
header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   16                                                  
NAXIS2  =                  387                                                  
EXTEND  =                    T                                                  
COL01DEF= 'wavelength'         / Label for data in column 1                     
COL02DEF= 'flux    '           / Label for data in column 2                     
COL03DEF= 'error (RMS+SYS)'    / Label for data in column 3                     
COL04DEF= 'error (RMS)'        / Label for data in column 4                     
COL05DEF= 'error (SYS)'        / Label for data in column 5                     
COL06DEF= 'offset uncertainty (CAL)' / Label for data in column 6               
COL07DEF= 'sky     '        

In [201]:
# Can query header
header['RA_SLT']  # Right ascension.
header['DEC_SLT']  # Declination.
header['BUNIT']  # Units of flux (typically Jy)
header['AORKEY']  # Unique observation identifier (specific to Spitzer Space Telescope).
header['OBJECT']  # Object name.

'HD 99754'

In [202]:
# Can convert to other formats if we want to use the header information for something.
# See http://docs.astropy.org/en/stable/io/fits/api/headers.html

# header_str = header.tostring()
# header.totextfile('test_header.csv')

***

# Appendix B -- Extra meta, ignore for now

### The associated metadata .tbl (table) file

In [19]:
test_meta = meta_dir + test_spec.split('/')[-1].split('.fits')[0] + '.tbl'
test_meta

'metadata/10019072.tbl'

In [20]:
# Ensure the metadata file exists..
os.path.isfile(test_meta)

True

In [28]:
meta_table = Table.read(test_meta, format='ascii.ipac')
meta_table

reqkey,bcdid,modedisplayname,wavelength,cra,cdec,exposuretime,primaryfield,filetype,externalname,ptcomment,epoch,equinox,scet,minwavelength,maxwavelength,fovname,dceid,campid,filesize,ra,dec,ra1,dec1,ra2,dec2,ra3,dec3,ra4,dec4,naxis1,naxis2,cdelt1,cdelt2,crota2,crpix1,crpix2,crval1,crval2,ctype1,ctype2,heritagefilename,hasAccess,accessUrl,accessWithAnc1Url
int64,int64,str9,str23,str12,str12,float64,int64,str5,str60,str222,float64,float64,str19,float64,float64,str35,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,str12,str12,str89,str4,str104,str117
10019072,327037996,IRS Stare,IRS LL 14.0-38.0um,11h28m31.38s,-23d50m30.9s,31.46,1,Image,r10019072/ch2/bcd/SPITZER_S2_10019072_0008_0000_11_bcd.fits,"128x128 32-bit (real) collapsed (2D) slope image CORRECTED for stray light (in case of SL) or crosstalk (in case of SH and LH), after division by the flatfield image. Obtained via ramp-slope estimation in the IRS pipeline.",2004.4859,2000.0,2004-06-25 20:09:17,14.0,38.0,IRS_Long-Lo_2nd_Order_1st_Position,15866521,738,83520,172.1308,-23.8419,172.1804,-23.863,172.1789,-23.8657,172.0812,-23.8208,172.0827,-23.8182,2,2,-0.0015,0.05,63.3417,1.5,1.5,172.1308,-23.8419,RA---TAN,DEC--TAN,/sha/archive/proc/IRSX003400/r10019072/ch2/bcd/SPITZER_S2_10019072_0008_0000_11_bcd.fits,true,http://sha.ipac.caltech.edu/applications/Spitzer/SHA/servlet/ProductDownload?DATASET=level1&ID=327037996,http://sha.ipac.caltech.edu/applications/Spitzer/SHA/servlet/ProductDownload?DATASET=level1&ID=327037996&OPTIONS=anc1
10019072,327038004,IRS Stare,IRS LL 14.0-38.0um,11h28m18.94s,-23d49m5.1s,31.46,1,Image,r10019072/ch2/bcd/SPITZER_S2_10019072_0010_0002_11_bcd.fits,"128x128 32-bit (real) collapsed (2D) slope image CORRECTED for stray light (in case of SL) or crosstalk (in case of SH and LH), after division by the flatfield image. Obtained via ramp-slope estimation in the IRS pipeline.",2004.4859,2000.0,2004-06-25 20:12:25,14.0,38.0,IRS_Long-Lo_1st_Order_1st_Position,15866525,738,83520,172.0789,-23.8181,172.1285,-23.8392,172.127,-23.8418,172.0294,-23.797,172.0308,-23.7943,2,2,-0.0015,0.05,63.3406,1.5,1.5,172.0789,-23.8181,RA---TAN,DEC--TAN,/sha/archive/proc/IRSX003400/r10019072/ch2/bcd/SPITZER_S2_10019072_0010_0002_11_bcd.fits,true,http://sha.ipac.caltech.edu/applications/Spitzer/SHA/servlet/ProductDownload?DATASET=level1&ID=327038004,http://sha.ipac.caltech.edu/applications/Spitzer/SHA/servlet/ProductDownload?DATASET=level1&ID=327038004&OPTIONS=anc1
10019072,327038008,IRS Stare,IRS LL 14.0-38.0um,11h28m15.33s,-23d48m40.2s,31.46,1,Image,r10019072/ch2/bcd/SPITZER_S2_10019072_0011_0003_11_bcd.fits,"128x128 32-bit (real) collapsed (2D) slope image CORRECTED for stray light (in case of SL) or crosstalk (in case of SH and LH), after division by the flatfield image. Obtained via ramp-slope estimation in the IRS pipeline.",2004.4859,2000.0,2004-06-25 20:15:55,14.0,38.0,IRS_Long-Lo_1st_Order_2nd_Position,15866530,738,83520,172.0639,-23.8112,172.1135,-23.8323,172.112,-23.8349,172.0143,-23.79,172.0158,-23.7874,2,2,-0.0015,0.05,63.3366,1.5,1.5,172.0639,-23.8112,RA---TAN,DEC--TAN,/sha/archive/proc/IRSX003400/r10019072/ch2/bcd/SPITZER_S2_10019072_0011_0003_11_bcd.fits,true,http://sha.ipac.caltech.edu/applications/Spitzer/SHA/servlet/ProductDownload?DATASET=level1&ID=327038008,http://sha.ipac.caltech.edu/applications/Spitzer/SHA/servlet/ProductDownload?DATASET=level1&ID=327038008&OPTIONS=anc1
10019072,327038420,IRS Stare,IRS SH 9.9-19.6um,11h28m23.30s,-23d49m34.3s,31.46,1,Image,r10019072/ch1/bcd/SPITZER_S1_10019072_0006_0000_11_bcd.fits,"128x128 32-bit (real) collapsed (2D) slope image CORRECTED for stray light (in case of SL) or crosstalk (in case of SH and LH), after division by the flatfield image. Obtained via ramp-slope estimation in the IRS pipeline.",2004.4859,2000.0,2004-06-25 20:05:40,9.9,19.6,IRS_Short-Hi_1st_Position,15866517,738,86400,172.0971,-23.8262,172